In [1]:
@decorate
def target():
    print('running target()')

NameError: name 'decorate' is not defined

In [ ]:
def target():
    print('running target()')
    
target = decorate(target)

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [3]:
@deco
def target():
    print('running target()')

In [4]:
target()

running inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

In [8]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x00000246ABFAC708>)
running register(<function f2 at 0x00000246AC0D4CA8>)
running main()
registry -> [<function f1 at 0x00000246ABFAC708>, <function f2 at 0x00000246AC0D4CA8>]
running f1()
running f2()
running f3()


In [10]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """选择可用的最佳折扣"""
    return max(promo(order) for promo in promos)

In [11]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [12]:
b = 6
f1(3)

3
6


In [13]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [14]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [15]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [16]:
f3(3)

3
6


In [17]:
b

9

In [18]:
f3(3)

3
9


In [19]:
b = 30

In [20]:
b

30

In [21]:
from dis import dis

In [22]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [23]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [24]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [25]:
avg = Averager()

In [26]:
avg(10)

10.0

In [27]:
avg(11)

10.5

In [28]:
avg(12)

11.0

In [29]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [31]:
avg = make_averager()

In [32]:
avg(10)

10.0

In [33]:
avg(11)

10.5

In [34]:
avg(12)

11.0

In [36]:
avg.__code__.co_varnames

('new_value', 'total')

In [37]:
avg.__code__.co_freevars

('series',)

In [38]:
avg.__closure__

(<cell at 0x00000246AC517108: list object at 0x00000246AC124A88>,)

In [39]:
avg.__code__.co_freevars

('series',)

In [42]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [43]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

In [44]:
avg = make_averager()

In [45]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [46]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

In [47]:
avg = make_averager()

In [48]:
avg(10)

10.0

In [49]:
avg(11)

10.5

In [50]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [51]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12354480s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000040s] factorial(1) -> 1
[0.00003500s] factorial(2) -> 2
[0.00004590s] factorial(3) -> 6
[0.00005500s] factorial(4) -> 24
[0.00006390s] factorial(5) -> 120
[0.00007470s] factorial(6) -> 720
6! =  720


In [52]:
import time
from functools import wraps

def clock(func):
    @wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [53]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [54]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [55]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [56]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [57]:
htmlize(42)

'<pre>42</pre>'

In [58]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [59]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [1]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x0000014C267CC798>)
running main()
registry -> [<function f1 at 0x0000014C267CC798>]
running f1()


In [4]:
registry = set()
def register(active=True):
    def decorate(func):
        print('running register(active=%s )->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False )->decorate(<function f1 at 0x0000014C275EE828>)
running register(active=True )->decorate(<function f2 at 0x0000014C275EE438>)


In [5]:
registry

{<function __main__.f2()>}

In [6]:
register()(f3)

running register(active=True )->decorate(<function f3 at 0x0000014C275EE4C8>)


<function __main__.f3()>

In [7]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [10]:
register(active=False)(f3)

running register(active=False )->decorate(<function f3 at 0x0000014C275EE4C8>)


<function __main__.f3()>

In [11]:
registry

{<function __main__.f2()>}

In [14]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ", ".join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        snooze(.123)

[0.12328148s] snooze(0.123) -> None
[0.12369442s] snooze(0.123) -> None
[0.12362432s] snooze(0.123) -> None


In [15]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12300252914428711s
snooze: 0.12314867973327637s
snooze: 0.1238546371459961s


In [16]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
